In [1]:
import sys
sys.path.insert(0,'/home/gustavu92/Desktop/unicamp/tools/')
sys.path.insert(0,'/home/gustavu92/Desktop/unicamp/models/unet/')

import glob
import natsort
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import load_model
import multiprocessing
import matplotlib.pyplot as plt
import numpy as np
import cv2

from data_generator import DataGenerator
from evaluation import metric_generator
from evaluation import filter_path
from evaluation import define_metrics

/home/gustavu92/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [3]:
metrics_list = define_metrics()
metrics_dict = {}
for metric in metrics_list:
    metrics_dict[metric.__name__] = metric

In [4]:
model = load_model('/home/gustavu92/Desktop/unicamp/models/unet/bonn/best_model_255.h5', custom_objects=metrics_dict)
# model = load_model('/home/gustavu92/Desktop/unicamp/models/unet/bonn/best_model_prep.h5', custom_objects=metrics_dict)

W0927 08:17:56.244931 140516312958784 deprecation_wrapper.py:119] From /home/gustavu92/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0927 08:17:56.250658 140516312958784 deprecation_wrapper.py:119] From /home/gustavu92/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0927 08:17:56.272512 140516312958784 deprecation_wrapper.py:119] From /home/gustavu92/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0927 08:17:56.339203 140516312958784 deprecation_wrapper.py:119] From /home/gustavu92/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph inste

In [5]:
# images_path = glob.glob('/mnt/1058CF1419A58A26/Bonn2016/*/images/rgb/*.png')
# annotations_path = glob.glob('/mnt/1058CF1419A58A26/Bonn2016/*/annotations/dlp/colorCleaned/*.png')

# images_path = natsort.natsorted(images_path)
# annotations_path = natsort.natsorted(annotations_path)

# images_path, annotations_path = filter_path(images_path, annotations_path)

# images_path = pandas.DataFrame(images_path, columns=['path'])
# annotations_path = pandas.DataFrame(annotations_path, columns=['path'])

In [6]:
X_train = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/X_train.pkl')
X_val = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/X_val.pkl')
X_test = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/X_test.pkl')
y_train = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/y_train.pkl')
y_val = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/y_val.pkl')
y_test = pd.read_pickle('/home/gustavu92/Desktop/unicamp/data/bonn/y_test.pkl')

In [7]:
validation_generator = DataGenerator(X_val, y_val, load_memory=False,  aug_flag=False, batch_size=4)
test_generator = DataGenerator(X_test, y_test, load_memory=False,  aug_flag=False, batch_size=4)

# validation_generator = DataGenerator(X_val, y_val, load_memory=False, preprocessing = 'norm',  aug_flag=False, batch_size=4)
# test_generator = DataGenerator(X_test, y_test, load_memory=False, preprocessing = 'norm',  aug_flag=False, batch_size=4)

In [8]:
# val_pred = model.predict_generator(generator=validation_generator,
#                                    use_multiprocessing=True,
#                                    workers=multiprocessing.cpu_count() - 1,
#                                    max_q_size=100)

In [9]:
val_metrics = model.evaluate_generator(generator=validation_generator,
                                       use_multiprocessing=True,
                                       workers=multiprocessing.cpu_count() - 1,
                                       max_q_size=100)

/home/gustavu92/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `evaluate_generator` call to the Keras 2 API: `evaluate_generator(generator=<data_gene..., use_multiprocessing=True, workers=5, max_queue_size=100)`
  after removing the cwd from sys.path.


In [10]:
['%s: %.4f'%(name,value) for name, value in zip(list(metrics_dict.keys()), val_metrics[1:])]

['total_acc: 0.9865',
 'total_precision: 0.0496',
 'total_recall: 0.0365',
 'total_iou: 0.0265',
 'total_f1: 0.0407',
 'crop_acc: 0.9864',
 'crop_precision: 0.1894',
 'crop_recall: 0.0358',
 'crop_iou: 0.0356',
 'crop_f1: 0.0516',
 'weed_acc: 0.9866',
 'weed_precision: 0.0013',
 'weed_recall: 0.0016',
 'weed_iou: 0.0007',
 'weed_f1: 0.0012']

In [11]:
test_metrics = model.evaluate_generator(generator=test_generator,
                                       use_multiprocessing=True,
                                       workers=multiprocessing.cpu_count() - 1,
                                       max_q_size=100)

/home/gustavu92/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `evaluate_generator` call to the Keras 2 API: `evaluate_generator(generator=<data_gene..., use_multiprocessing=True, workers=5, max_queue_size=100)`
  after removing the cwd from sys.path.
Process ForkPoolWorker-12:
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-14:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/pyt

KeyboardInterrupt: 

In [ ]:
['%s: %.4f'%(name,value) for name, value in zip(list(metrics_dict.keys()), test_metrics[1:])]

In [ ]:
plt.imshow(np.squeeze(pred[...,0]>0.5))

In [ ]:
dim=(512, 384)
for i in range(50):
    img = cv2.resize(cv2.imread(X_test.iloc[i]['path']), dim)
    mask = cv2.imread(y_test.iloc[i]['path'])
    pred = model.predict(np.expand_dims(img, 0)/255)
    fig, ax = plt.subplots(1,3)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    ax[0].imshow(img[...,::-1])
    ax[1].imshow(mask)
    ax[2].imshow(np.concatenate((np.zeros_like(pred[0,...,:1]),pred[0,...]), axis=-1))
    plt.show()